## EDA Juan

Configuracion y carga de tablas

In [ ]:
import pandas as pd
import numpy as np
import os

# Usamos la ruta de la carpeta que contiene los csv
PATH_DATA = 'C:/Users/sebas/OneDrive/Escritorio/Henry/csvpf/' 

# Carga de los csv
try:
    df_summary = pd.read_csv(os.path.join(PATH_DATA, 'game_summary.csv'))
    df_line = pd.read_csv(os.path.join(PATH_DATA, 'line_score.csv'))
    df_inactive = pd.read_csv(os.path.join(PATH_DATA, 'inactive_players.csv'))
    df_info = pd.read_csv(os.path.join(PATH_DATA, 'game_info.csv'))
    print("✅ Archivos base cargados exitosamente.")
except FileNotFoundError as e:
    print(f"⚠️ Error al cargar los archivos. Revisar la ruta: {PATH_DATA}. Detalle: {e}")

✅ Archivos base cargados exitosamente.


Limpieza de tablas (game_summary y line_score)

In [3]:
print("\n--- PASO 1: LIMPIEZA DE TABLAS CORE ---")

# --- 1.1 Limpieza y el tipo de dato de df_summary (fechas y IDs) ---
# Convertir la columna de fecha a tipo datetime (vital para el análisis temporal)
df_summary['game_date_est'] = pd.to_datetime(df_summary['game_date_est'], errors='coerce')

# Seleccionar solo columnas esenciales para el merge y el análisis
df_summary = df_summary[['game_id', 'game_date_est', 'home_team_id', 'visitor_team_id', 'season']].copy()
# Eliminar filas con IDs de juego nulos (no se pueden rastrear)
df_summary.dropna(subset=['game_id'], inplace=True)


# --- 1.2 Limpieza de df_line (puntuaciones de tiempo extra) ---
# Identificar columnas de tiempos extra (pts_ot1_home, pts_ot2_home, etc.)
ot_cols_home = [col for col in df_line.columns if 'pts_ot' in col and '_home' in col]
ot_cols_away = [col for col in df_line.columns if 'pts_ot' in col and '_away' in col]

# Reemplazar nulos en OT con 0.0, ya que NaN en esta columna significa 0 puntos
df_line[ot_cols_home + ot_cols_away] = df_line[ot_cols_home + ot_cols_away].fillna(0.0)

# Seleccionar solo las columnas necesarias para el merge
df_line = df_line[['game_id', 'pts_home', 'pts_away']].copy()

print(f"✅ game_summary y line_score limpiados. Puntuaciones OT nulas convertidas a 0.")


--- PASO 1: LIMPIEZA DE TABLAS CORE ---
✅ game_summary y line_score limpiados. Puntuaciones OT nulas convertidas a 0.


Creacion de la variable Target y Spread

Estas variable spueden ser usadas por el modelo

In [4]:
print("\n--- PASO 2: CREACIÓN DE TARGET Y SPREAD ---")

# --- 2.1 Unión de Tablas Centrales ---
# Merge basado en la clave primaria 'game_id'
df_master = pd.merge(df_summary, df_line, on='game_id', how='inner')

# --- 2.2 Creación de Variable Target (Outcome) ---
# HOME_WINS: 1 si el equipo local gana, 0 si pierde
df_master['HOME_WINS'] = np.where(df_master['pts_home'] > df_master['pts_away'], 1, 0)
# (NOTA: Asumimos que no hay empates en los datos de puntuación final de la NBA)

# --- 2.3 Creación del Spread (SCORE_DIFF) ---
# Marcas de apuestas clave: el margen de victoria del equipo local
df_master['SCORE_DIFF'] = df_master['pts_home'] - df_master['pts_away']

# Verificación de la integridad del Target
win_balance = df_master['HOME_WINS'].value_counts(normalize=True).mul(100).round(2)
print(f"📊 Balance de la Variable Target (Victoria Local):\n{win_balance}")
print(f"✅ Variable target HOME_WINS y SCORE_DIFF (spread) creadas.")


--- PASO 2: CREACIÓN DE TARGET Y SPREAD ---
📊 Balance de la Variable Target (Victoria Local):
HOME_WINS
1    50.99
0    49.01
Name: proportion, dtype: float64
✅ Variable target HOME_WINS y SCORE_DIFF (spread) creadas.


Jugadores Inactivos

Cantidad de juagdores inactivos por equipo/juego. Muestra la fuerza del equipo para ese juego

In [5]:
print("\n--- PASO 3: JUGADORES (INACTIVOS) ---")

# --- 3.1 Conteo por Juego y Equipo ---
# Agrupar el archivo 'inactive_players' para obtener el conteo de ausencias por juego/equipo
df_inactive_count = df_inactive.groupby(['game_id', 'team_id']).size().reset_index(name='N_INACTIVE_PLAYERS')

# --- 3.2 Preparación para Merge ---
# Renombrar para hacer el merge con el ID del equipo local (HOME)
df_inactive_home = df_inactive_count.rename(columns={'team_id': 'home_team_id', 
                                                     'N_INACTIVE_PLAYERS': 'HOME_N_INACTIVE'})
# Renombrar para hacer el merge con el ID del equipo visitante (VISITOR)
df_inactive_visitor = df_inactive_count.rename(columns={'team_id': 'visitor_team_id', 
                                                        'N_INACTIVE_PLAYERS': 'VISITOR_N_INACTIVE'})

# --- 3.3 Unión y Relleno de Nulos ---
# Merge con la tabla maestra
df_master = pd.merge(df_master, df_inactive_home, on=['game_id', 'home_team_id'], how='left')
df_master = pd.merge(df_master, df_inactive_visitor, on=['game_id', 'visitor_team_id'], how='left')

# Si un juego/equipo no está en 'inactive_players', significa que el conteo es 0.
df_master[['HOME_N_INACTIVE', 'VISITOR_N_INACTIVE']] = df_master[['HOME_N_INACTIVE', 'VISITOR_N_INACTIVE']].fillna(0)

print("✅ Conteo de jugadores inactivos agregado. Nulos reemplazados por 0.")


--- PASO 3: JUGADORES (INACTIVOS) ---
✅ Conteo de jugadores inactivos agregado. Nulos reemplazados por 0.


Integracion del contexto

Asistencia del publico (attendance) es un factor que ayuda a la ventaja del local

In [6]:
print("\n--- PASO 4: INTEGRACIÓN DE CONTEXTO Y REVISIÓN FINAL ---")

# --- 4.1 Limpieza de game_info ---
# Asistencia es el factor más relevante de este archivo para la predicción
df_info['attendance'] = df_info['attendance'].fillna(0) 

# Seleccionar solo columnas relevantes
df_info_slim = df_info[['game_id', 'attendance']].copy()

# --- 4.2 Unión Final ---
df_master = pd.merge(df_master, df_info_slim, on='game_id', how='left')

# --- 4.3 Revisión Final de la Tabla Maestra ---
print(f"Filas de la Tabla Maestra: {df_master.shape[0]}")
print(f"Columnas de la Tabla Maestra: {df_master.shape[1]}")

print("\n--- df_master.head() (Muestra del resultado final) ---")
print(df_master.head())

# Verifica si el dtype de las features es el correcto antes de pasar a Big Query
print("\n--- df_master.info() (Verificación de Tipos) ---")
df_master.info()


--- PASO 4: INTEGRACIÓN DE CONTEXTO Y REVISIÓN FINAL ---
Filas de la Tabla Maestra: 58342
Columnas de la Tabla Maestra: 12

--- df_master.head() (Muestra del resultado final) ---
    game_id game_date_est  home_team_id  visitor_team_id  season  pts_home  \
0  24600001    1946-11-01    1610610035       1610612752    1946      66.0   
1  24600003    1946-11-02    1610610034       1610610031    1946      56.0   
2  24600002    1946-11-02    1610610032       1610612738    1946      53.0   
3  24600004    1946-11-02    1610610025       1610612752    1946      63.0   
4  24600005    1946-11-02    1610610028       1610610036    1946      50.0   

   pts_away  HOME_WINS  SCORE_DIFF  HOME_N_INACTIVE  VISITOR_N_INACTIVE  \
0      68.0          0        -2.0              0.0                 0.0   
1      51.0          1         5.0              0.0                 0.0   
2      59.0          0        -6.0              0.0                 0.0   
3      47.0          1        16.0              0.0

Creacion de carpeta y export del archivo de la tabla master con los datos relevantes de los otros 4 archivos (csv) de la base original

In [ ]:
import os
# Asegúrate de que df_master es el DataFrame final de tu script EDA tocaria juntar los EDA de los demas en esta

# 1. Definir la ruta de salida. Creamos una carpeta 'processed_data'
OUTPUT_PATH = 'C:/Users/sebas/OneDrive/Escritorio/Henry/processed_data/'
OUTPUT_FILENAME = 'df_master_analitica.csv'

# 2. Crear la carpeta si no existe (Best Practice: asegura que el script no falle)
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 3. Exportar a CSV
# index=False: No incluimos el índice de Pandas. ¡Esto es crucial para datos limpios!
# encoding='utf-8': Asegura la compatibilidad universal del archivo
df_master.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILENAME), index=False, encoding='utf-8')

print(f"\n✅ Exportación Exitosa:")
print(f"El archivo '{OUTPUT_FILENAME}' se ha guardado en: {OUTPUT_PATH}")


✅ Exportación Exitosa:
El archivo 'df_master_analitica.csv' se ha guardado en: C:/Users/sebas/OneDrive/Escritorio/Henry/PF-HENRY-GRUPO01/processed_data/
